# 📘 Архитектуры Агентов 22: Contract Net Protocol (Аукцион Задач)

В этом блокноте мы реализуем протокол **Contract Net** — распределенную архитектуру решения задач, где распределение работы происходит через механизм переговоров (аукциона). 

Вместо того чтобы жестко прописывать, какой агент что делает, "Менеджер" объявляет тендер на задачу. "Подрядчики" (специализированные агенты) оценивают свои возможности и делают ставки (Bids), указывая цену, время и качество. Менеджер выбирает лучшее предложение.

### Определение
**Contract Net Protocol (CNP)** состоит из следующих фаз:
1. **Task Announcement (Объявление задачи):** Менеджер транслирует спецификацию задачи всем доступным агентам.
2. **Bidding (Торги):** Агенты анализируют задачу и, если могут её выполнить, отправляют предложение (Bid) с параметрами (цена, скорость, точность).
3. **Awarding (Награждение):** Менеджер оценивает полученные предложения и заключает "контракт" с победителем.
4. **Execution (Выполнение):** Победитель выполняет задачу и возвращает результат.

### Применения
*   **Распределенные вычисления:** Балансировка нагрузки между серверами.
*   **Логистика:** Выбор курьера для доставки (кто ближе/быстрее).
*   **Маркетплейс агентов:** Динамический выбор лучшей модели (GPT-4 vs Haiku vs Llama) под конкретный промпт в зависимости от бюджета.

## Фаза 0: Основа и Настройка

### Шаг 0.1: Установка библиотек

**Что мы будем делать:**
Установим стандартный стек библиотек. Нам не нужны специфические инструменты, только LangSmith/LangGraph.

In [ ]:
# !pip install -q -U langchain-nebius langchain langgraph rich python-dotenv

### Шаг 0.2: Настройка окружения

**Что мы будем делать:**
Настроим ключи и модель. В данном случае одной модели LLM хватит для симуляции всех участников (Менеджера и Подрядчиков), мы будем просто менять системные промпты.

In [ ]:
import os
import json
from typing import List, Dict, Any, Optional, Literal, TypedDict
from dotenv import load_dotenv
from pydantic import BaseModel, Field

from langchain_nebius import ChatNebius
from langgraph.graph import StateGraph, END

from rich.console import Console
from rich.panel import Panel
from rich.table import Table

from notebook_utils import setup_environment

console = setup_environment(
    project_name="Agentic Architecture - Contract Net (Nebius)",
    required_keys=["NEBIUS_API_KEY", "LANGCHAIN_API_KEY"]
)

llm = ChatNebius(model="meta-llama/Meta-Llama-3.1-8B-Instruct", temperature=0.1)

### Шаг 0.3: Модели протокола Contract Net

**Что мы будем делать:**
Определим форматы сообщений:
1. **TaskSpec:** Описание задачи (тип, сложность, дедлайн).
2. **Bid:** Предложение исполнителя (цена, время, уверенность).
3. **Contract:** Финальное соглашение (победитель).

In [ ]:
class TaskSpec(BaseModel):
    description: str = Field(description="Текстовое описание задачи")
    required_skills: List[str] = Field(description="Необходимые навыки")
    max_budget: float = Field(description="Максимальный бюджет ($)")
    deadline_ms: int = Field(description="Дедлайн (мс)")

class Bid(BaseModel):
    agent_name: str = Field(description="Имя подрядчика")
    cost: float = Field(description="Запрашиваемая цена ($)")
    estimated_time: int = Field(description="Оценка времени (мс)")
    quality_score: float = Field(description="Ожидаемое качество (0-100)")
    justification: str = Field(description="Почему стоит выбрать меня")

class Contract(BaseModel):
    winner_name: str = Field(description="Имя победителя")
    final_cost: float = Field(description="Согласованная цена")
    reasoning: str = Field(description="Причина выбора")

class CNPState(TypedDict):
    user_request: str
    task_spec: Optional[Dict]
    bids: List[Dict]
    contract: Optional[Dict]
    final_result: Optional[str]

## Фаза 1: Роли Агентов

Создадим "личности" для Менеджера и нескольких Подрядчиков.

### Шаг 1.1: Менеджер (Manager)

**Что мы будем делать:**
Реализуем узлы:
1. **`announce_task_node`**: Превращает запрос пользователя в формальную `TaskSpec`.
2. **`evaluate_bids_node`**: Выбирает победную ставку на основе критериев (например, лучшее соотношение цены/качества).

In [ ]:
def announce_task_node(state: CNPState):
    console.print("--- [MANAGER] Creating Task Spec --- ")
    runner = llm.with_structured_output(TaskSpec)
    spec = runner.invoke(f"Create a formal task specification for: {state['user_request']}")
    
    console.print(Panel(spec.model_dump_json(indent=2), title="Task Spec", border_style="blue"))
    return {"task_spec": spec.model_dump()}

def evaluate_bids_node(state: CNPState):
    console.print(f"--- [MANAGER] Evaluating {len(state['bids'])} bids --- ")
    
    runner = llm.with_structured_output(Contract)
    
    prompt = f"""
    Task: {state['task_spec']}
    
    Bids:
    {json.dumps(state['bids'], indent=2)}
    
    Select the best bid based on optimal trade-off between Cost, Time, and Quality.
    """
    
    contract = runner.invoke(prompt)
    console.print(Panel(
        f"Winner: [bold green]{contract.winner_name}[/bold green]\n"
        f"Cost: ${contract.final_cost}\n"
        f"Reason: {contract.reasoning}",
        title="Contract Awarded",
        border_style="gold1"
    ))
    return {"contract": contract.model_dump()}

### Шаг 1.2: Подрядчики (Contractors)

**Что мы будем делать:**
Создадим универсальную функцию `contractor_node`, которая принимает профиль агента (например, "Быстрый но дорогой" или "Дешевый стажер").
Агент оценивает задачу и, если она соответствует его навыкам, делает ставку.

In [ ]:
def make_contractor_node(name: str, profile: str):
    def contractor_logic(state: CNPState):
        task = state['task_spec']
        
        runner = llm.with_structured_output(Bid)
        prompt = f"""
        You are agent '{name}'. Profile: {profile}
        
        Task: {task['description']}
        Budget: {task['max_budget']}
        Skills needed: {task['required_skills']}
        
        Generate a bid based on your profile.
        If you cannot do it, set quality=0.
        """
        
        bid = runner.invoke(prompt)
        
        color = "cyan" if "Fast" in name else "magenta" if "Cheap" in name else "white"
        console.print(f"[{color}]Agent {name} bids: ${bid.cost}, {bid.estimated_time}ms, Q:{bid.quality_score}[/{color}]")
        
        # Добавляем ставку к существующим
        current_bids = state.get('bids', [])
        return {"bids": current_bids + [bid.model_dump()]}
    
    return contractor_logic

## Фаза 2: Сборка и Запуск

Соберем граф. Все подрядчики могут работать параллельно после объявления задачи.

### Шаг 2.1: Конфигурация Графа

**Что мы будем делать:**
1. Создадим трех разных подрядчиков:
   - **TurboCoder:** Быстрый, дорогой, среднее качество.
   - **BudgetDev:** Очень дешевый, медленный, низкое качество.
   - **SeniorArch:** Высочайшее качество, очень дорого, умеренная скорость.
2. Настроим параллельное выполнение их узлов.

In [ ]:
workflow = StateGraph(CNPState)

# Узлы
workflow.add_node("manager_announce", announce_task_node)
workflow.add_node("worker_1", make_contractor_node("TurboCoder", "Fast execution, high price, good quality. Expert in Python."))
workflow.add_node("worker_2", make_contractor_node("BudgetDev", "Low price, slow execution, junior quality. Knows basics."))
workflow.add_node("worker_3", make_contractor_node("SeniorArch", "Very high price, meticulous quality, takes time. Expert Architect."))
workflow.add_node("manager_evaluate", evaluate_bids_node)

# Ребра
workflow.set_entry_point("manager_announce")

# Фан-аут: после анонса запускаем всех воркеров
workflow.add_edge("manager_announce", "worker_1")
workflow.add_edge("manager_announce", "worker_2")
workflow.add_edge("manager_announce", "worker_3")

# Фан-ин: все воркеры идут к оценке
workflow.add_edge("worker_1", "manager_evaluate")
workflow.add_edge("worker_2", "manager_evaluate")
workflow.add_edge("worker_3", "manager_evaluate")

workflow.add_edge("manager_evaluate", END)

app = workflow.compile()

### Шаг 2.2: Запуск Демонстрации

**Что мы будем делать:**
Попробуем разные задачи, чтобы увидеть, кого выберет менеджер.
1. Критически важная архитектурная задача (ожидаем SeniorArch).
2. Простой скрипт для переименования файлов (ожидаем BudgetDev).

In [ ]:
tasks = [
    "Design a high-load microservices architecture for a banking system processing 1M TPS.",
    "Write a simple script to rename all .txt files to .md in a folder."
]

for task in tasks:
    console.print(f"\n[bold white on blue] Request: {task} [/bold white on blue]")
    app.invoke({"user_request": task, "bids": []})

## Заключение

Протокол Contract Net позволяет создавать **рыночные механизмы** внутри агентных систем. Это идеально подходит для оптимизации ресурсов, когда у вас есть выбор между несколькими моделями или инструментами с разными характеристиками (цена/качество/скорость).